__применение парсера объявлений авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
avito_path = 'sevastopol/kvartiry/prodam'

raw_data_path = 'data/raw/' # каталог с файлами объявлений
loc_file_path = 'data/location.pkl' # таблица адресов
result_file_path = 'data/data.pkl'

In [6]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
ts

'2022-09-07_12-39'

## качаем новые данные

In [7]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True) #, page_limit=4)

print(len(df))

[INFO    ] 2022-09-07 12:39:22 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-07 12:39:22 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-07 12:39:24 | AdsListParser: downloader init
[INFO    ] 2022-09-07 12:39:24 | AvitoParser: init
[INFO    ] 2022-09-07 12:39:29 | AvitoParser: 79 pages for read
[INFO    ] 2022-09-07 12:39:29 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-07 12:39:32 | AdsListParser: read page 1
[INFO    ] 2022-09-07 12:39:35 | AdsListParser: read page 2
[INFO    ] 2022-09-07 12:39:38 | AdsListParser: read page 3
[INFO    ] 2022-09-07 12:39:41 | AdsListParser: read page 4
[INFO    ] 2022-09-07 12:39:44 | AdsListParser: read page 5
[INFO    ] 2022-09-07 12:39:48 | AdsListParser: read page 6
[INFO    ] 2022-09-07 12:39:52 | AdsListParser: read page 7
[INFO    ] 2022-09-07 12:39:55 | AdsListParser: read page 8
[INFO    ] 2022-09-07 12:39:59 | AdsListParser: read page 9
[INFO    ] 2022-09-07 12:40:03 | AdsListParser

In [ ]:
print(len(df))
df = df.dropna().reset_index(drop=True)
print(len(df))

In [9]:
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [10]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts
874,2055843092,"Объявление «1-к. апартаменты, 23,5 м², 1/5 эт.» 16 фотографий",6100000,,"ул. Челюскинцев, 57/2","Продаются апартаменты у моря и шикарного парка «Учкуевка». Дом новый. Полуцоколь. \n\nВысота потолков 3.4 м! Отделка сделана недавно, новая мебель, холодильник, микроволновка, мультиварка, вытяжка, бойлер, стиральная машинка, кондиционер зима/лето, теплые по",18,2022-09-07 12:46:11.483203
1146,1971022174,"Объявление «4-к. квартира, 80 м², 1/3 эт.» 23 фотографии",11500000,,"ул. Новикова, 5","Потрясающая великолепием и простором 4х комнатная сталинка. Находится в самом Центре Балаклавы(Жемчужина Крыма с ласковым морем). Напротив нашего дома детский оборудованый Парк, школа 3 минуты пешком, детский сад 4 минуты пешком, администрация, дом культур",23,2022-09-07 12:46:11.483203


In [11]:
# html[-1]
# with open('tmp/avito.html','wt') as f: f.write(html[1])

## собираем и чистим данные

In [12]:
import re
from os import listdir

raw_data_files = sorted([ raw_data_path+'/'+f for f in listdir(raw_data_path) if re.match(r'.+\.xlsx$',f) ])
raw_data_files

['data/raw//avito_2022-08-26_17-29_raw.xlsx',
 'data/raw//avito_2022-08-29_13-16_raw.xlsx',
 'data/raw//avito_2022-08-31_17-42_raw.xlsx',
 'data/raw//avito_2022-09-01_14-07_raw.xlsx',
 'data/raw//avito_2022-09-02_13-04_raw.xlsx',
 'data/raw//avito_2022-09-05_14-02_raw.xlsx',
 'data/raw//avito_2022-09-06_13-09_raw.xlsx',
 'data/raw//avito_2022-09-07_12-39_raw.xlsx']

In [13]:
from lib.avito import AvitoDataCleanerRealty

data = pd.concat(
        [ pd.read_excel(f) for f in raw_data_files ]
    ).drop_duplicates().reset_index(drop=True)

data = AvitoDataCleanerRealty.transform( data )

print(len(data))

29536


In [14]:
assert len(data)>0

In [15]:
data.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29536 entries, 0 to 29535
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   avito_id       29536 non-null  int64         
 1   title          29536 non-null  object        
 2   price          29536 non-null  int64         
 3   adr            29536 non-null  object        
 4   description    29512 non-null  object        
 5   obj_name       17664 non-null  object        
 6   ts             29536 non-null  datetime64[ns]
 7   avito_page     25677 non-null  float64       
 8   page           3859 non-null   float64       
 9   nrooms         29536 non-null  int64         
 10  floor          29536 non-null  int64         
 11  nfloors        29536 non-null  int64         
 12  area           29536 non-null  float64       
 13  is_studio      29536 non-null  bool          
 14  is_apartment   29536 non-null  bool          
 15  is_part        2953

### обновляем таблицу адресов

In [16]:
from lib.locator import LocationUpdater
from lib.locator import AddressTransformerSev

loc = LocationUpdater(
        address_transformer=AddressTransformerSev(),
    ).transform(
        adr=data['adr'],
        loc=pd.read_pickle(loc_file_path),
        show_pbar=True,
    )

[INFO    ] 2022-09-07 12:47:27 | LocationUpdater: 1837 addresses in location table
[INFO    ] 2022-09-07 12:47:27 | LocationUpdater: 1901 addresses total
[INFO    ] 2022-09-07 12:47:27 | LocationUpdater: 1837 addresses defined
[INFO    ] 2022-09-07 12:47:27 | LocationUpdater: 64 addresses undefined


  0%|          | 0/64 [00:00<?, ?it/s]

[INFO    ] 2022-09-07 12:48:35 | LocationUpdater: 18 new addresses found


In [17]:
loc.sample(3)

,adr,latitude,longitude,truncated
1166,"Качинское шоссе , 35Б",44.66,33.55,False
877,"улица Суворова, 31",44.61,33.52,False
1492,"Балаклава, улица Урицкого, 3",44.51,33.60,False


In [18]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

### дополняем данные геометкой

In [19]:
data = data.merge(loc[['adr','latitude','longitude',]],on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isnull() ] ) )

всего записей: 29536
записей без геометки: 1372


In [20]:
assert len(data)>0
data.to_pickle(result_file_path)

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29536 entries, 0 to 29535
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   avito_id       29536 non-null  int64         
 1   title          29536 non-null  object        
 2   price          29536 non-null  int64         
 3   adr            29536 non-null  object        
 4   description    29512 non-null  object        
 5   obj_name       17664 non-null  object        
 6   ts             29536 non-null  datetime64[ns]
 7   avito_page     25677 non-null  float64       
 8   page           3859 non-null   float64       
 9   nrooms         29536 non-null  int64         
 10  floor          29536 non-null  int64         
 11  nfloors        29536 non-null  int64         
 12  area           29536 non-null  float64       
 13  is_studio      29536 non-null  bool          
 14  is_apartment   29536 non-null  bool          
 15  is_part        2953

In [ ]:
# for f in raw_data_files:
#     ts = dtm.strptime( re.sub(r'.*/avito_','',f), '%Y-%m-%d_%H-%M_raw.xlsx')
#     df = pd.read_excel(f)
#     df['ts'] = ts
#     df.to_excel(f,index=False)
#     print(ts)